In [ ]:
# Uninstall existing Pytorch on Colab, which might be incompatible or buggy.
!pip uninstall --yes torch torchvision torchaudio torchtext
# Install Pytorch 1.8.2 LTS (Long Term Support) and auto_LiRPA. It might take a few minutes depending on network speed.
!pip install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install git+https://github.com/KaidiXu/auto_LiRPA.git
# Clear installation output to avoid clutter.
from IPython.display import clear_output
clear_output()


In [ ]:
pip install torch==2.0.1


In [ ]:
!git clone https://github.com/stanleybak/vnncomp2021.git

Cloning into 'vnncomp2021'...
remote: Enumerating objects: 5827, done.
remote: Total 5827 (delta 0), reused 0 (delta 0), pack-reused 5827
Receiving objects: 100% (5827/5827), 98.54 MiB | 11.21 MiB/s, done.
Resolving deltas: 100% (1861/1861), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/alpha-beta-CROWN-main/complete_verifier")

In [ ]:
!pip install onnxruntime
!pip install onnx2pytorch
!pip install onnxoptimizer
!pip install skl2onnx
clear_output()

In [ ]:
%env AUTOLIRPA_DEBUG_GRAPH=1

Calling ABCrown.py with neuralhash yaml file and posponed attacks.

In [ ]:
!python abcrown.py --config exp_configs/tutorial_examples/nhash_custom_old.yaml --pgd_order=after

/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr

Additional Code Snippets for images processing and generating hash


In [ ]:
# TODO: NeuralHash Hash Generation Documentation
import torch
import numpy as np
import onnx
from PIL import Image
from torchvision.datasets import CIFAR10
from neuralhash import NeuralHash

# Define transformations for resizing and normalizing the images.
def transform_image(image):
    # Resize the image to the target size
    image = image.resize((360, 360))
    # Convert the image to a NumPy array and normalize the pixel values
    image_np = np.array(image).astype(np.float32) / 255.0
    # Transpose dimensions to match the input shape (assuming channels-last order)
    image_np = image_np.transpose((2, 0, 1))
    # Convert the NumPy array to a PyTorch tensor
    image_tensor = torch.from_numpy(image_np)
    # Add a batch dimension (assuming a batch size of 1)
    image_tensor = image_tensor.unsqueeze(0)
    # print(image_tensor.shape)
    return image_tensor

def load_cifar10_dataset():
    dataset = CIFAR10(root='data', train=True,
                      download=True, transform=transform_image)
    return dataset

In [ ]:
def load_hash_matrix(path='custom/neuralhash_128x96_seed1.dat'):
    with open(path, 'rb') as file:
        seed1_data = file.read()[128:]

    seed1_data = np.frombuffer(seed1_data, dtype=np.float32)
    seed1_data = seed1_data.reshape([96, 128])
    return seed1_data

def generate_hash(model, image_tensor):
    model.eval()
    seed = load_hash_matrix()
    hash_hex = ""
    # Compute the hash output by multiplying the hash matrix with the flattened model output on its binary representation and convert to hexadecimal format
    with torch.no_grad():
        output = model(image_tensor)
        hash_output = seed.dot(output[0].flatten())
        hash_bits = ''.join(['1' if it >= 0 else '0' for it in hash_output])
        hash_hex = '{:0{}x}'.format(int(hash_bits, 2), len(hash_bits) // 4)
    return hash_hex

In [ ]:
# Generate Hash
onnx_model = onnx.load('neuralhash.onnx')
neural_hash_model = NeuralHash(onnx_model)

# Load the cifar10 Dataset with image preprocess
dataset = load_cifar10_dataset()
hashes = []
# Generate the hash for the image
for img in dataset:
  hash = generate_hash(neural_hash_model,img[0])
  print(hash)
  hashes.append(hash)

print("Generated Hash List:", hashes)

All parameters assigned
Files already downloaded and verified
34f6bb0b912d6ec551b55f68
3ce3479bb9313ad988b51334
f3f2ac9ff7f9cbe19c4b1734
3443a39bbf714f5b98ff16e4
967ace9b09314b41984b4724
96b82b5a494d6880404bfa0d
b476504781c6f9df2d33eab1
901aa1712986460d78a0b334
01b7212032d6051f58a833bc
5068a1afba5149388c0b76f6
10f74059cdcd3873d344af54
fec38cffd0189a7b987c9cd3
fcf09e5de7920c09a0848571
e048631b1f600be5dac3f0f3
16916a5d99303ac381947136
85ad8878fe2caf36ede2dfab
5392abd36119a8e27965f2b9
e6e763be8e5c4d63da9919f8
d12984994e97cfe5c5b084a5
34f7491d04757a5f08b51c94
7fab81f0e28793175de0bf4d
cfbfcc5028079332ba61e63c
bbbfc203e4f5b149d8d123a0
35b3e659cd25dc41d5888326
11f6aa4dc9642bc051116a38
bd21767d7f70a4e9d1e48626
56d26b47417ae9c005771f14
1610fa1d5d003a0de18c7197
5dd84e78b9101a1be2c57427
d16c80adf32ec3e5dd3226e5
dc0f2d2bbb18c1b9e6f57b3d
04e9a8cbbb2e87326d63ffa1
1df6332999b910e45f514368
343540990e64e61735b68c8a
98aaacaafa0e9736be40be07
3d670ecfb87d74ef3347c30a
76f749908c019a17d2740d70
3cf46c4a890d9